Load libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Ochtend

Load en bekijk data

In [3]:
# load the data in
df = pd.read_csv('titanic_data.csv')

Bekijk de datatypes

In [4]:
# view only the head of the data 
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Wat is de minimum, maximum en gemiddelde leeftijd van de passagiers?

In [5]:
print("This is the minimum {}".format(df['Age'].min()))
print("This is the mean {}".format(df['Age'].mean()))
print("This is the maximum {}".format(df['Age'].max()))

This is the minimum 0.42
This is the mean 29.69911764705882
This is the maximum 80.0


Wat is het percentage van mensen die de ramp niet hebben overleefd?

In [6]:
survived = df['Survived']

survivers = (survived == 1).sum()
victims = (survived == 0).sum()

precentVictims = victims / (victims + survivers) * 100

print("The precent of victim is {}".format(precentVictims))

The precent of victim is 61.61616161616161


Wat is de minimum, gemiddelde, en maximum prijs (Fare) in de verschillende klassen (Pclass)?

In [32]:
grouped = df.groupby('Pclass')

stats = grouped['Fare'].agg(['min', 'mean', 'max'])

stats

,min,mean,max
Pclass,,,
1,0.0,84.154687,512.3292
2,0.0,20.662183,73.5000
3,0.0,13.675550,69.5500


Welke data is categorisch?

###### Name, Sex,  Cabin and Ticket

Verijder de kolommen 'Name', 'Cabin' en 'Ticket'

In [40]:
new_df = df.drop(['Name', 'Cabin', 'Ticket'], axis=1)

new_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


Codeer categorische features met indices (getallen)

In [43]:
#factorize Name and team columns only
df[['Name', 'Sex', 'Ticket']] = df[['Name', 'Sex', 'Ticket']].apply(lambda x: pd.factorize(x)[0])

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,0,0,22.0,1,0,0,7.2500,NaN,S
1,2,1,1,1,1,38.0,1,0,1,71.2833,C85,C
2,3,1,3,2,1,26.0,0,0,2,7.9250,NaN,S
3,4,1,1,3,1,35.0,1,0,3,53.1000,C123,S
4,5,0,3,4,0,35.0,0,0,4,8.0500,NaN,S


Vervang NaN in "Ages". Bepaal zelf of mediaan of gemiddelde hiervoor het best is.

In [54]:
# I choose to replace the NaN in "Ages" with the median
# Because there is some outliers that would affect the results

df['Age'].fillna(d'f['Age'].median()', inplace=True)

df

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1654107493.py, line 4)

## Middag

Voorspel of passagiers de ramp hebben overleefd of niet. Bereid de data voor om je model te trainen. Wat wordt je target vector en hoe ziet je feature matrix er uit?

Train en test KNN classifier

Train en test regressie classifier. Kies je lineair of logistisch?

Welk model geeft het beste resultaat?

Extra: Kan je het resultaat verbeteren?